notebook required packages:

- request
- rioxarray
- matplotlib

# Trasformation Framework 

## Overview

The ESA Transformation Framework is a component of the
Copernicus Sentinels Collaborative Data Hub Software (DHS) intended to provide
data transformation capabilities via the integration of processing elements
applied on-demand to Copernicus Sentinel products.


#### Basic fuctionalities implemented for 0.8-osf release
The user will have the possibility to:  
  - request the list of available Workflow definitions
  - request the definition of a specific Workflow
  - submit a Transformation Order
  - request the list of all the Tranformations Orders
  - request a single Tranformation Order

#### Additional fuctionalities implemented for 0.8-osf release 
The TF will:
  - perform parallel processing with Dask
  - download product from a set of data source (defined in config/hubs_credentials.yaml)
  - not re-submit a Transformation Order with the same parameters of an other one currently enqueued, in processing or succefully completed.
  - re-submit on demand failed Transformations Orders
  - check the validity of input workflow parameters before triggering the processing
  - check the input product type before triggering the processing



## How to explore available Workflows 

The API endpoints will be available at http://localhost:8080 exposing a subset of Open Data Protocol, as long as the docker compose is running.


Alternatively it can be used: 
 - _curl_, from a command line interface  
 - _request_, from a python interface

####  How to get the list of available Workflow definition

The following URL can then be visited to display the list of available Workflows:
http://localhost:8080/Workflows

Alternatively:

In [ ]:
import requests

res = requests.get('http://localhost:8080/Workflows')
res.json()

#### How to get the definition of a specific Workflow: `sen2cor_l1c_l2a`

At following URL it is available: `sen2cor_l1c_l2a` Workflow: 
http://localhost:8080/Workflows('sen2cor_l1c_l2a')

In [ ]:
res = requests.get("http://localhost:8080/Workflows('sen2cor_l1c_l2a')")
res.json()

## How to submit a Transformation Order  


In order to request a new Order, it is possible to use the command _curl_ in the command-line interface (or _request.post_ in a python interface), passing through a JSON string (or a dictionary) with the full desired configuration, including the workflow id, the product id and the workflow options.

The Transformation Framework will:
- verify the type and range of the options passed by the User
- submit the processing to the dask workers

- create the processing directory
- download the product from the first hub in the _hubs_credentials.yaml_ that has published the product
- run the plugin workflow
- zip and move the output in the output directory.
- remove the processing directory



#### Sen2Cor 
The `sen2cor_l1c_l2a` plugin installed within the TF makes use of [Sen2Cor v2.9](https://step.esa.int/main/snap-supported-plugins/sen2cor/) tool to convert Sentinel-2 L1C products into L2A output product. It implements classification and atmospheric correction. Currently it is used the Digital Elevation Model of the Shuttle Radar Topography Mission [(SRTM DEM)](http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/), downloaded by Sen2Cor.


#### Acquisition on Paris: S2A_MSIL1C_20211231T105441_N0301_R051_T31UEP_20211231T125354

In [ ]:
params_paris = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference":  "S2B_MSIL1C_20211109T110159_N0301_R094_T31UDQ_20211109T114303", 
        "DataSourceName": "scihub"
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "RURAL", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": False, 
        "DEM_Terrain_Correction": False,
        "Resolution": 60,
    }
}

order_submission_paris = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_paris, 
)

order_submission_paris.json()

In [ ]:
order_id_paris = order_submission_paris.json()["Id"]
order_id_paris

## How to monitor the Tranformation Orders status

- The full list of available Transformation Orders can be retrieved via the following URL:

  http://localhost:8080/TransformationOrders

- The single Transformation Order can be retrieved using the following URL (where \<order_id\> is the ID of the Trasformation Oder):

   http://localhost:8080/TransformationOrders('\<order_id\>')

- It is also possible to filter accessible orders by status type. 

  For example the following URL queries the list of completed orders:

  http://localhost:8080/TransformationOrders?\$filter=Status%20eq%20'completed'

#### Request list of all Transformation Order

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders")
oders_status.json()

#### Request specific Transfromation Oreder

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id_paris}')")
oders_status.json()

## Example of completed Procesing

#### Acquisition on Rome: S2A_MSIL1C_20211216T100421_N0301_R122_T32TQM_20211216T105832

In [ ]:
params_rome = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference": "S2A_MSIL1C_20211216T100421_N0301_R122_T32TQM_20211216T105832",
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "RURAL", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": True, 
        "DEM_Terrain_Correction": True,
        "Resolution": 60,
    }
}

order_submission_rome = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_rome, 
)

order_submission_rome.json()

In [ ]:
order_id_rome = order_submission_rome.json()["Id"]
order_id_rome

Read Output filename

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id_rome}')")
output_filename = oders_status.json()["OutputFile"]

Unzip Product

In [ ]:
import glob
import os
! unzip {os.path.join("../esa_tf/output", output_filename)} -d tmp/

rgb_path = glob.glob(f"tmp/{output_filename[:-4]}.SAFE/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2")[0]

Read True Color Image and Plot

In [ ]:
import rioxarray
from matplotlib import pyplot as plt

rgb = rioxarray.open_rasterio(rgb_path)

plt.figure(figsize=(20, 20))
plt.imshow(rgb.transpose("y", "x", "band"))
plt.title("Rome: Sentinel2 L2A True Color 60m", fontsize="xx-large");

## Additional implemented features


#### The TF checks the validity of input workflow parameters before trigger the processing

In the following Workflow options example, `Cirrus_Correction` parameter has a wrong dtype, integer instead of bool. The TF raises the error before triggering the plugin processing.

In [ ]:
params_wrong_options = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference": "S2A_MSIL1C_20211022T062221_N0301_R048_T39GWH_20211022T064133.zip", 
        "DataSourceName": "apihub"
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "MARITIME", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": 1, 
        "DEM_Terrain_Correction": True
    }
}

In [ ]:
order_submission_wrong_options = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_wrong_options, 
)
order_submission_wrong_options.json()

#### The TF does not re-submit a Transformation Order with the same parameters of an other one currently enqueued, in processing or succefully completed.

Submit the Transformation Order

In [ ]:
order_submission = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params_paris, 
)

Monitor the submited Transformation Order

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders")
oders_status.json()

#### The TF re-submits on demand failed Transformation Order

Submit a Transformation Order with a typo in the product Name

In [ ]:
params = {
    "WorkflowId": "sen2cor_l1c_l2a", 
    "InputProductReference": {
        "Reference": "S2A_MSIL1C_20211022T062221_N0301_R048_T39GWH_20211022T064133.zip", 
        "DataSourceName": "apihub"
    }, 
    "WorkflowOptions": {
        "Aerosol_Type": "MARITIME", 
        "Mid_Latitude": "AUTO", 
        "Ozone_Content": 0, 
        "Cirrus_Correction": True, 
        "DEM_Terrain_Correction": True
    }
}

order_submission = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params, 
)
order_id = order_submission.json()["Id"]
order_submission.json()

Monitor the Transformation Order

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id}')")
oders_status.json()

Re-submit and monitor the Transformation Order

In [ ]:
order_submission = requests.post(
    "http://localhost:8080/TransformationOrders", 
    json=params, 
)

order_submission.json()

## Results on Paris

Read output filename

In [ ]:
oders_status = requests.get(f"http://localhost:8080/TransformationOrders('{order_id_paris}')")
oders_status.json()

In [ ]:
output_filename = oders_status.json()["OutputFile"]
output_filename

Unzip product

In [ ]:
! unzip {os.path.join("../esa_tf/output", output_filename)} -d tmp/ 

rgb_path = glob.glob(f"tmp/{output_filename[:-4]}.SAFE/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2")[0]

Read True Color Image and Plot

In [ ]:
rgb = rioxarray.open_rasterio(rgb_path)

plt.figure(figsize=(20, 20))
plt.imshow(rgb.transpose("y", "x", "band"))
plt.title("Rome: Sentinel2 L2A True Color 60m", fontsize="xx-large");